In [ ]:
import pandas as pd
import numpy as np

seeds = [f's{i}' for i in range(1, 11)]
experiments = ['baseline', 'integrated_1', 'integrated_2', 'integrated_3']
tests_score = {}
run_stats = {}

for experiment in experiments:
    tests_data = f'../../datalake/data_{experiment}/data/xdpole/runstats'
    tests_score[experiment] = []
    run_stats[experiment] = []
    for seed in seeds:
        test = pd.read_csv(f'{tests_data}/{seed}_test.csv')
        tests_score[experiment].append(test.score[0])

        stats = pd.read_csv(f'{tests_data}/{seed}_run.csv')
        run_stats[experiment].append(stats.avgfit)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

df = pd.DataFrame(tests_score)
sns.boxplot(df, flierprops={"marker": "x"})
plt.title("Experiments Post-Evaluation Performance")
plt.ylabel("Performance")
plt.xlabel("Experiment")
plt.show()

In [ ]:
mean_stats = []

for experiment in experiments:
    df = pd.DataFrame(run_stats[experiment])
    df_t = df.T
    df_t.columns = seeds
    df_mean = [np.mean(row) for row in df_t.values]
    mean_stats.append(df_mean)

df = pd.DataFrame(mean_stats)
df = df.T
df.columns = experiments
sns.relplot(df, kind='line')

In [ ]:
general = pd.DataFrame()
limit = 4000

for seed in seeds:
    baseline_data = f'../../datalake/data_baseline/data/xdpole/baseconditions'
    bdf = pd.read_csv(f'{baseline_data}/{seed}_run.csv')
    integrated_data = f'../../datalake/data_integrated_3/data/xdpole/baseconditions'
    idf = pd.read_csv(f'{integrated_data}/{seed}_run.csv')

    diff = idf[:limit] - bdf[:limit]
    general = diff if general.empty else general + diff

general = general / len(seed)

sns.heatmap(general.T, vmin=-1000, vmax=1000, cmap="vlag_r")
plt.title("Conditions Grid Evaluation")
plt.ylabel("Set")
plt.xlabel("Generation")
plt.show()